In [33]:
import pandas as pd
import numpy as np


In [34]:
df = pd.read_csv("weatherHistory.csv")
df.head()


,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [35]:
# select useful columns
data = df[['Formatted Date','Temperature (C)','Humidity']]
data['Formatted Date'] = pd.to_datetime(data['Formatted Date'])
data = data.sort_values('Formatted Date').reset_index(drop=True)



C:\Users\Aero15-OLED Gaming\AppData\Local\Temp\ipykernel_15592\1673966459.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Formatted Date'] = pd.to_datetime(data['Formatted Date'])
C:\Users\Aero15-OLED Gaming\AppData\Local\Temp\ipykernel_15592\1673966459.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Formatted Date'] = pd.to_datetime(data['Formatted Date'])


In [36]:
# discretize into categorical observations
def temp_to_cat(t):
    if t < 5: return "Cold"
    elif t < 20: return "Mild"
    else: return "Hot"

def hum_to_cat(h):
    if h < 0.4: return "Dry"
    elif h < 0.7: return "Normal"
    else: return "Humid"

obs = data.apply(lambda row: temp_to_cat(row['Temperature (C)']) + "_" + hum_to_cat(row['Humidity']), axis=1)
obs = obs.tolist()


In [37]:
# unique observations mapping
unique_obs = list(set(obs))
obs_index = {o:i for i,o in enumerate(unique_obs)}
obs_seq = [obs_index[o] for o in obs]


In [38]:
# HMM setup
states = ["Sunny","Rainy"]
N = len(states)
M = len(unique_obs)

np.random.seed(42)
start_prob = np.full(N, 1.0/N)
trans_prob = np.random.dirichlet(np.ones(N), size=N)
emit_prob = np.random.dirichlet(np.ones(M), size=N)


In [39]:
# Baum-Welch training
def forward(obs_seq, start_prob, trans_prob, emit_prob):
    T = len(obs_seq)
    N = len(start_prob)
    alpha = np.zeros((T,N))
    alpha[0] = start_prob * emit_prob[:, obs_seq[0]]
    for t in range(1,T):
        for j in range(N):
            alpha[t,j] = np.sum(alpha[t-1] * trans_prob[:,j]) * emit_prob[j, obs_seq[t]]
    return alpha

def backward(obs_seq, trans_prob, emit_prob):
    T = len(obs_seq)
    N = trans_prob.shape[0]
    beta = np.zeros((T,N))
    beta[T-1] = np.ones(N)
    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t,i] = np.sum(trans_prob[i,:] * emit_prob[:,obs_seq[t+1]] * beta[t+1])
    return beta

def baum_welch(obs_seq, start_prob, trans_prob, emit_prob, iterations=10):
    N = len(start_prob)
    M = emit_prob.shape[1]
    T = len(obs_seq)
    for _ in range(iterations):
        alpha = forward(obs_seq, start_prob, trans_prob, emit_prob)
        beta = backward(obs_seq, trans_prob, emit_prob)
        xi = np.zeros((T-1,N,N))
        for t in range(T-1):
            denom = np.sum(alpha[t] @ trans_prob * emit_prob[:,obs_seq[t+1]] * beta[t+1])
            for i in range(N):
                numer = alpha[t,i] * trans_prob[i,:] * emit_prob[:,obs_seq[t+1]] * beta[t+1]
                xi[t,i,:] = numer / denom
        gamma = np.sum(xi, axis=2)
        start_prob = gamma[0] / np.sum(gamma[0])
        trans_prob = np.sum(xi, axis=0) / np.sum(gamma, axis=0)[:,None]
        gamma_full = np.vstack((gamma, np.sum(xi[-1], axis=1)))
        for j in range(M):
            mask = np.array([1 if obs_seq[t]==j else 0 for t in range(T)])
            emit_prob[:,j] = np.sum(gamma_full * mask[:,None], axis=0)
        emit_prob = emit_prob / np.sum(emit_prob, axis=1)[:,None]
    return start_prob, trans_prob, emit_prob


In [40]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=2, n_iter=20, random_state=42)
obs_seq = np.array(obs_seq).reshape(-1,1)
model.fit(obs_seq)
path = model.predict(obs_seq)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [41]:
# Viterbi
def viterbi(obs_seq, start_prob, trans_prob, emit_prob):
    T = len(obs_seq)
    N = len(start_prob)
    V = np.zeros((T,N))
    B = np.zeros((T,N), dtype=int)
    V[0] = np.log(start_prob) + np.log(emit_prob[:, obs_seq[0]])
    for t in range(1,T):
        for s in range(N):
            prev = V[t-1] + np.log(trans_prob[:,s])
            B[t,s] = np.argmax(prev)
            V[t,s] = np.max(prev) + np.log(emit_prob[s, obs_seq[t]])
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(V[-1])
    for t in range(T-2,-1,-1):
        path[t] = B[t+1, path[t+1]]
    prob = np.exp(np.max(V[-1]))
    return path, prob


In [42]:
path, prob = viterbi(obs_seq, start_prob, trans_prob, emit_prob)
decoded_states = [states[i] for i in path]


ValueError: could not broadcast input array from shape (2,2) into shape (2,)

In [ ]:
print("Sample predictions:")
for i in range(10):
    print(data['Formatted Date'].iloc[i], "Observation:", obs[i], "→ State:", decoded_states[i])
print("Sequence probability:", prob)


Sample predictions:
2006-01-01 00:00:00+01:00 Observation: Cold_Humid → State: Sunny
2006-01-01 01:00:00+01:00 Observation: Cold_Humid → State: Rainy
2006-01-01 02:00:00+01:00 Observation: Cold_Humid → State: Sunny
2006-01-01 03:00:00+01:00 Observation: Cold_Humid → State: Rainy
2006-01-01 04:00:00+01:00 Observation: Cold_Humid → State: Sunny
2006-01-01 05:00:00+01:00 Observation: Cold_Humid → State: Rainy
2006-01-01 06:00:00+01:00 Observation: Cold_Humid → State: Sunny
2006-01-01 07:00:00+01:00 Observation: Cold_Humid → State: Rainy
2006-01-01 08:00:00+01:00 Observation: Cold_Humid → State: Sunny
2006-01-01 09:00:00+01:00 Observation: Cold_Humid → State: Rainy
Sequence probability: 0.0


In [ ]:
last_state = path[-1]
forecast_probs = trans_prob[last_state]
print("Forecast for next day:")
for i,s in enumerate(states):
    print(s, ":", forecast_probs[i])


Forecast for next day:
Sunny : 0.13487081200849088
Rainy : 0.8651291879915091
